In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ]
)

# レスポンスの表示
pprint(vars(response))

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

# 役割や前提の設定
role = "あなたはマーケティング分野に精通したビジネスコンサルタントです。企業の成長をサポートするために、効果的なマーケティング戦略を提供します。"

# メッセージの設定
message = "新製品の発売に向けた効果的なマーケティング戦略を教えてください。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": message},
    ],
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())


# 出力の多様性
message = "日本の魅力を紹介して！"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    temperature=1.4,
    presence_penalty=-2.0,
    n = 3, # 3回答を得る
    max_completion_tokens = 100 # 出力トークン長を制限
)

# 結果を表示（複数回答を比較）
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

{'_request_id': 'req_6e29a8097739431d8e14bf21b757f1e6',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='言語モデルを使用する際のポイントはいくつかあります。以下に示す点を考慮することで、効果的にモデルを活用できます。\n\n1. **目的の明確化**: 言語モデルを使用する目的を明確にします。例えば、テキスト生成、要約、翻訳、質問応答など、用途によってアプローチが異なる場合があります。\n\n2. **プロンプトの工夫**: 言語モデルは入力に対する応答を生成します。そのため、プロンプト（入力文）の設計が重要です。具体的かつ明確な指示を与えることで、より良い結果を得られます。\n\n3. **コンテキストの提供**: モデルに与える情報の量や質が応答に影響します。必要な背景情報を提供することで、より関連性の高い出力が得られます。\n\n4. **フィードバックの活用**: モデルが生成した結果に対してフィードバックを行い、プロンプトを調整することで、より良い応答を引き出すことができます。\n\n5. **多様性の確保**: 同じプロンプトを使った場合、モデルは常に同じ出力を生成するわけではありません。異なる実行や設定を試すことで、さまざまな視点やアイデアを得ることができます。\n\n6. **倫理的な考慮**: 言語モデルを使用する際には、出力が倫理的であるか、バイアスが含まれていないかを注意深く確認することが重要です。\n\n7. **限界の理解**: 言語モデルには限界があります。特定の知識が不足している場合や誤情報を生成することがあるため、その点に留意し、自動生成された情報を過信しないことが重要です。\n\n8. **継続的な学習**: 言語モデルや関連技術は進化しています。最新の研究やトレンドを追うことで、より効果的にモデルを活用することができます。\n\nこれらのポイントを考慮することで、言語モデルをより効果的に活用できるでしょう。', refusal=None, role='assistant', annotations=[], audio

In [2]:
import tiktoken
from tiktoken.core import Encoding

# OpenAI APIの特定のモデルに対応するトークナイザーを取得
encoding: Encoding = tiktoken.encoding_for_model(MODEL_NAME) # o200k_base

# テキストをトークンIDのリストに変換
tokens = encoding.encode("こんにちは！")
tokens_count = len(tokens)

# トークンの長さとトークンIDを表示
print(f"{tokens_count=}")
print(f"{tokens=}")

# トークンの選択度合い
message = """
AさんとBさんで会話してください。
A:あ、Bさんだ。
B:
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    max_tokens=100,
    n=3,
    logit_bias = {95839:8, 3393:8}
)

# 結果を表示
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

tokens_count=2
tokens=[95839, 3393]
--------------------
B:こんにちは！Aさん、どうしていましたか？
--------------------
B:こんにちは！Aさん、最近どうですか？
--------------------
B:こんにちは！Aさん、元気でしたか？
言語モデルを効果的に活用するためのポイントはいくつかあります。以下に主な点を挙げます。

1. **明確な目的を持つ**:
   - 使用する目的を明確にし、その目的に応じたプロンプトを設計します。例えば、情報検索、文章生成、要約など目的によってアプローチが変わります。

2. **具体的なプロンプトの設計**:
   - 入力する質問や命令が具体的であるほど、モデルから得られる回答も明確になります。なるべく詳細に指示を与えることが効果的です。

3. **コンテキストを提供する**:
   - 以前の会話や関連情報を含めることで、モデルがより適切な回答を生成しやすくなります。文脈を提供することで一貫性のある応答が得られます。

4. **トーンとスタイルの調整**:
   - 必要に応じて、モデルに特定のトーンやスタイルで回答を求めることができます。たとえば、「カジュアル」や「フォーマル」な表現を指定することができます。

5. **反復とフィードバック**:
   - モデルから得られた結果に基づいて、プロンプトを調整し、繰り返し試行することで、より良い結果を得られることが多いです。

6. **倫理とバイアスへの配慮**:
   - モデルが出力する内容には、偏見や不適切な情報が含まれる可能性があります。出力を批判的に評価し、倫理的に問題がないか確認することが重要です。

7. **最新の情報を確認**:
   - モデルは訓練データに基づいているため、最新の情報や出来事については正確性が欠ける場合があります。重要な情報を確認する際は、最新の情報源を利用することが必要です。

8. **創造的なアプローチ**:
   - モデルは創造性を発揮するためのツールとしても利用できます。アイデアのブレインストーミングやストーリーの構築に活用することができます。

これらのポイントを踏まえ、言語モデルを効果的に活用することで、さまざまなタスクを支援す